<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/FaceFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Install Dependencies**
import os

# Clone repository
!git clone -q https://github.com/qsardor/Deepfake roop > /dev/null 2>&1
%cd roop

# Set CUDA paths (mandatory for GPU detection)
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:/usr/lib/x86_64-linux-gnu"
os.environ["PATH"] += ":/usr/local/cuda/bin"

# Install dependencies quietly
!pip install --quiet --no-warn-conflicts -r requirements.txt --force-reinstall > /dev/null 2>&1

print("Installation done")

In [ ]:
#@title **Run Job (Google Drive)**
import os
from google.colab import drive, files
from IPython.display import display, Image

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define paths
DRIVE_DIR = "/content/drive/MyDrive/Deepfake"
INPUT_DIR = os.path.join(DRIVE_DIR, "input")
OUTPUT_DIR = os.path.join(DRIVE_DIR, "output")
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Check/Upload files
source_path = os.path.join(INPUT_DIR, "source.png")
target_files = [f for f in os.listdir(INPUT_DIR) if f.startswith("target.")]

if not os.path.exists(source_path):
    print("Uploading source.png...")
    files.upload()
if not target_files:
    print("Uploading target file (image/video)...")
    files.upload()
    target_files = [f for f in os.listdir(INPUT_DIR) if f.startswith("target.")]

# Detect target type
target_file = os.path.join(INPUT_DIR, target_files[0])
output_ext = "png" if target_file.endswith(".png") else "mp4"
output_path = os.path.join(OUTPUT_DIR, f"result.{output_ext}")

# Set CUDA environment variables
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:/usr/lib/x86_64-linux-gnu"
os.environ["PATH"] += ":/usr/local/cuda/bin"

# Run processing
%cd /content/roop
!python run.py -s "{source_path}" -t "{target_file}" -o "{output_path}" --frame-processor face_swapper face_enhancer --keep-frames --keep-fps

# Show result
if os.path.exists(output_path):
    if output_ext == "png":
        display(Image(output_path))
    else:
        print(f"Video saved to {output_path}")
    print(f"Result saved to: {output_path}")
else:
    print("Processing failed. Check input files and GPU runtime.")

In [ ]:
#@title **Run Job (Local Files)**
import os
from google.colab import files
from IPython.display import display, Image

# Set CUDA paths (critical for GPU detection)
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:/usr/lib/x86_64-linux-gnu"
os.environ["PATH"] += ":/usr/local/cuda/bin"

# Check/Upload files
if not os.path.exists("source.png"):
    print("Uploading source.png...")
    files.upload()
if not os.path.exists("target.png") and not os.path.exists("target.mp4"):
    print("Uploading target file...")
    files.upload()

# Detect target type
target_file = "target.png" if os.path.exists("target.png") else "target.mp4"
output_ext = "png" if target_file.endswith(".png") else "mp4"
output_path = f"result.{output_ext}"

# Run with explicit GPU provider
%cd /content/roop
!python run.py -s "source.png" -t "{target_file}" -o "{output_path}" --frame-processor face_swapper face_enhancer --keep-frames --keep-fps

# Show result
if os.path.exists(output_path):
    display(Image(output_path)) if output_ext == "png" else print(f"Video saved to {output_path}")
    print(f"Result saved to: /content/roop/{output_path}")
else:
    print("Processing failed. Check CUDA configuration.")